In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [2]:
import gc
import os

In [3]:
data_dir = "../ml-25m/"
# movies
movies_df = pd.read_csv(data_dir+"movies.csv")
links_df = pd.read_csv(data_dir+"links.csv")
scores_df = pd.read_csv(data_dir+"genome-scores.csv")
tag_df = pd.read_csv(data_dir+"genome-tags.csv")

# user
ratings_df = pd.read_csv(data_dir+ "ratings.csv")
tags_df = pd.read_csv(data_dir+"tags.csv")

In [4]:
movies_df[:3]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [5]:
ratings_df[:3]

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828


In [6]:
links_df[:3]

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0


In [7]:
tags_df[:3]

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598


In [8]:
scores_df[:3]

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250


In [9]:
tag_df[:3]

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century


In [10]:
tmp1 = ratings_df.groupby("userId").count()['rating'].min()
tmp2 = ratings_df.groupby("movieId").count()['rating'].min()
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 1


In [11]:
tmp1 = sum(ratings_df.groupby("movieId").count()['rating'] == 1)
tmp2 = ratings_df['movieId'].drop_duplicates().count()
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

10298 out of 59047 movies are rated by only one user


In [12]:
del tmp1,tmp2

## show the number of user

In [13]:
# user_number
user_count = ratings_df['userId'].drop_duplicates().count()
user_count

162541

In [14]:
del user_count

## show the number of movie

In [15]:
# movie_number
movie_count = movies_df['movieId'].drop_duplicates().count()
movie_count

62423

In [16]:
del movie_count

In [17]:
movies_df['movieId'].max()

209171

## show the number of movie rated

In [18]:
movie_rated = movies_df['movieId'].isin(ratings_df).count()
movie_rated

62423

In [19]:
del movie_rated

# check if there are missing values in rating as well here

In [20]:
movie_missing_value = ratings_df[ratings_df.isnull().T.any()]
movie_missing_value

,userId,movieId,rating,timestamp


In [21]:
del movie_missing_value

# movie genres

In [22]:
movie_genres = movies_df['genres']
movie_genres[:10]

0    Adventure|Animation|Children|Comedy|Fantasy
1                     Adventure|Children|Fantasy
2                                 Comedy|Romance
3                           Comedy|Drama|Romance
4                                         Comedy
5                          Action|Crime|Thriller
6                                 Comedy|Romance
7                             Adventure|Children
8                                         Action
9                      Action|Adventure|Thriller
Name: genres, dtype: object

In [23]:
movie_genres=movies_df.drop('genres', axis=1).join(movies_df['genres'].str.split('|', expand=True).stack().reset_index(level=1, drop=True).rename('splited_genres'))
movie_genres[:10]

,movieId,title,splited_genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy
1,2,Jumanji (1995),Adventure
1,2,Jumanji (1995),Children
1,2,Jumanji (1995),Fantasy
2,3,Grumpier Old Men (1995),Comedy
2,3,Grumpier Old Men (1995),Romance


In [24]:
category_movie_count  = movie_genres.splited_genres.value_counts()  
print("Number of categories:",len(category_movie_count))
category_movie_count[:10]

Number of categories: 20


Drama                 25606
Comedy                16870
Thriller               8654
Romance                7719
Action                 7348
Horror                 5989
Documentary            5605
Crime                  5319
(no genres listed)     5062
Adventure              4145
Name: splited_genres, dtype: int64

In [25]:
del movie_genres

In [34]:
data = ratings_df.drop_duplicates()
# data = pd.merge(data,ratings_df)
data[:10]

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
5,1,1088,4.0,1147868495
6,1,1175,3.5,1147868826
7,1,1217,3.5,1147878326
8,1,1237,5.0,1147868839
9,1,1250,4.0,1147868414


In [27]:
len(data)

25000095

In [28]:
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss

DeepCTR version 0.8.3 detected. Your version is 0.8.2.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.8.3


In [29]:
# def split(x):
#     key_ans = x.split('|')
#     for key in key_ans:
#         if key not in key2index:
#             # Notice : input value 0 is a special "padding",so we do not use 0 to encode valid feature for sequence input
#             key2index[key] = len(key2index) + 1
#     return list(map(lambda x: key2index[x], key_ans))

# key2index = {}
# genres_list = list(map(split, data['genres'].values))
# genres_length = np.array(list(map(len, genres_list)))
# max_len = max(genres_length)
# # Notice : padding=`post`
# genres_list = pad_sequences(genres_list, maxlen=max_len, padding='post', )
# genres_list

In [31]:
gc.collect()

42

In [35]:
# Computer is out of memory. Fetch some data
data = data[:10000]
data

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
9995,75,736,4.0,1537207939
9996,75,778,3.0,1537208100
9997,75,783,3.0,1537348896
9998,75,805,3.5,1537348872


## train， test， prediction 

In [36]:
if __name__ == "__main__":
    sparse_features = ["movieId", "userId"]
    SEQ_LEN = 50


    # 1.Label Encoding for sparse features,and process sequence features with `gen_date_set` and `gen_model_input`

    features = ['userId', 'movieId']
    feature_max_idx = {}
    for feature in features:
        lbe = LabelEncoder()
        data[feature] = lbe.fit_transform(data[feature]) + 1
        feature_max_idx[feature] = data[feature].max() + 1

    user_profile = data[["userId"]].drop_duplicates('userId')

    item_profile = data[["movieId"]].drop_duplicates('movieId')

    user_profile.set_index("userId", inplace=True)

    user_item_list = data.groupby("userId")['movieId'].apply(list)

    train_set, test_set = gen_data_set(data, 0)

    train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
    test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)

    # 2.count #unique features for each sparse field and generate feature config for sequence feature

    embedding_dim = 16

    user_feature_columns = [SparseFeat('userId', feature_max_idx['userId'], embedding_dim),
                            VarLenSparseFeat(SparseFeat('hist_movie_id', feature_max_idx['movieId'], embedding_dim,
                                                        embedding_name="movieId"), SEQ_LEN, 'mean', 'hist_len'),]

    item_feature_columns = [SparseFeat('movieId', feature_max_idx['movieId'], embedding_dim)]

    # 3.Define Model and train

    K.set_learning_phase(True)

    import tensorflow as tf
    if tf.__version__ >= '2.0.0':
       tf.compat.v1.disable_eager_execution()
    
    model = YoutubeDNN(user_feature_columns, item_feature_columns, num_sampled=5, user_dnn_hidden_units=(64, embedding_dim))
    # model = MIND(user_feature_columns,item_feature_columns,dynamic_k=False,p=1,k_max=2,num_sampled=5,user_dnn_hidden_units=(64,embedding_dim),init_std=0.001)

    model.compile(optimizer="adam", loss=sampledsoftmaxloss)  # "binary_crossentropy")

    history = model.fit(train_model_input, train_label,  # train_label,
                        batch_size=256, epochs=1, verbose=1, validation_split=0.0, )

    # 4. Generate user features for testing and full item features for retrieval
    test_user_model_input = test_model_input
    all_item_model_input = {"movieId": item_profile['movieId'].values}
    
    
    user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
    item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)


    user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
    # user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND

    print(user_embs.shape)
    

    item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)
    print(item_embs.shape)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:00<00:00, 1233.90it/s]


6 6
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Train on 9850 samples
9850/9850 [==============================] - 1s 150us/sample - loss: 1.2379
(75, 16)
(3287, 16)


In [ ]:
# all_item_model_input
# test_user_model_input

In [40]:
user_embs

array([[-0.06975035,  0.06024284,  0.07555481, ..., -0.11581381,
        -0.03438999, -0.08401693],
       [-0.06836987,  0.05975564,  0.07558642, ..., -0.1135235 ,
        -0.03448129, -0.08261339],
       [-0.05858491,  0.05343557,  0.05701777, ..., -0.09442975,
        -0.035398  , -0.0720423 ],
       ...,
       [-0.06191878,  0.05289735,  0.07050627, ..., -0.10707679,
        -0.0323239 , -0.08345953],
       [-0.07497917,  0.06181774,  0.09091108, ..., -0.12939055,
        -0.03178284, -0.09316348],
       [-0.07057573,  0.06019191,  0.0764833 , ..., -0.11345545,
        -0.03150751, -0.08529839]], dtype=float32)

In [41]:
item_embs

array([[-2.8561059e-02,  2.9495571e-02,  2.7697125e-02, ...,
        -3.0655399e-02, -2.7822604e-02, -3.0143229e-02],
       [-1.2502024e-02,  1.2258508e-02,  1.2361874e-02, ...,
        -1.2517601e-02, -1.1702806e-02, -1.2071374e-02],
       [-4.2645088e-03,  3.8675012e-03,  4.0245363e-03, ...,
        -4.0331348e-03, -4.0505435e-03, -3.4334543e-03],
       ...,
       [-1.6537783e-04, -6.4743421e-05, -4.0685924e-04, ...,
        -1.2924569e-04,  2.6216634e-04, -2.4311227e-04],
       [-2.3979477e-03,  1.8583911e-03,  2.7431261e-03, ...,
        -1.4077420e-03, -1.8445444e-03, -1.3211920e-03],
       [ 4.9176104e-03, -4.7502215e-03, -4.5895842e-03, ...,
         4.4830530e-03,  4.8935455e-03,  4.2757341e-03]], dtype=float32)

In [58]:
    movies_df = pd.read_csv(data_dir+"movies.csv")
    movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [77]:
    #5. [Optional] Cosine similarity calculation
    
    from  sklearn.metrics import pairwise_distances
    user_item_similary = pairwise_distances(user_embs,item_embs,metric='euclidean')
    print("user_item_similary:",user_item_similary.shape)
    
    
    # test userId = 67
    import heapq
    user_67_similary = user_item_similary[66]
    
    user_67_top20 = heapq.nlargest(20, range(len(user_67_similary)), user_67_similary.__getitem__)
#     print(user_67_top20)
    user_result = movies_df.loc[movies_df["movieId"].isin(user_67_top20)]

    print("The recommended movie index for userId:67 is as follows:","\n", user_result)
    
    # test movie = 99
    from sklearn.metrics.pairwise import cosine_similarity
    movie_movie_similary = cosine_similarity(item_embs)
    movie_99_similary = movie_movie_similary[98]
    print("movie 99 shape:",movie_99_similary.shape)
    
    item_99_top20 = heapq.nlargest(20, range(len(movie_99_similary)), movie_99_similary.__getitem__)
    item_result = movies_df.loc[movies_df["movieId"].isin(item_99_top20)]
    

    


user_item_similary: (75, 3287)
The recommended movie index for userId:67 is as follows: 
       movieId                             title  \
160       162                      Crumb (1994)   
684       699       To Cross the Rubicon (1991)   
762       778              Trainspotting (1996)   
860       879                 Relic, The (1997)   
990      1013           Parent Trap, The (1961)   
1233     1266                 Unforgiven (1992)   
1290     1324      Amityville: Dollhouse (1996)   
1308     1343                  Cape Fear (1991)   
1912     2001            Lethal Weapon 2 (1989)   
2094     2184    Trouble with Harry, The (1955)   
2272     2363          Godzilla (Gojira) (1954)   
2340     2431                Patch Adams (1998)   
2391     2482                Still Crazy (1998)   
2502     2593  Monster, The (Mostro, Il) (1994)   
2507     2598                Pushing Tin (1999)   
2711     2803         Pelican Brief, The (1993)   
2823     2915             Risky Business (1

In [78]:
    print("A similar recommendation for movieId:66 :","\n",item_result)
    

A similar recommendation for movieId:66 : 
       movieId                                       title  \
84         85                   Angels and Insects (1995)   
96         98                             Shopping (1994)   
104       106  Nobody Loves Me (Keiner liebt mich) (1994)   
146       148            Awfully Big Adventure, An (1995)   
275       278                       Miami Rhapsody (1995)   
446       451                       Flesh and Bone (1993)   
539       544                    Striking Distance (1993)   
578       586                           Home Alone (1990)   
830       848                  Spitfire Grill, The (1996)   
960       982                               Picnic (1955)   
1005     1028                         Mary Poppins (1964)   
1018     1041                       Secrets & Lies (1996)   
1042     1067                Damsel in Distress, A (1937)   
1067     1094                     Crying Game, The (1992)   
1077     1104            Streetcar Named 